In [1]:
import solution_file_processing as sfp
import os
import numpy as np
import pandas as pd
from dask import dataframe as dd

from solution_file_processing.utils.utils import catch_errors
from solution_file_processing.utils.write_excel import write_xlsx_column, write_xlsx_stack, STACK_PALETTE, IEA_PALETTE_16, IEA_PALETTE_PLUS, EXTENDED_PALETTE
from solution_file_processing.constants import VRE_TECHS, PRETTY_MODEL_NAMES
from solution_file_processing.timeseries import create_output_11 as create_ts_output_11
from solution_file_processing.timeseries import create_output_4 as create_timeseries_output_4
from solution_file_processing import log
from solution_file_processing.plots import _get_plot_1_variables

# Initialize config with toml file
c = sfp.SolutionFilesConfig('config_archive/china/CHN.toml')

from solution_file_processing.plots import create_plot_1a

[18:16:20 00:00] I:117:__init__ - Logging to C:\Users\hart_c\showcase\solution-file-processing\logs\20240429_181620-CHN.log.
[18:16:20 00:00] I:149:__init__ - Initialized SolutionFilesConfig for config_archive/china/CHN.toml.


In [2]:
# would be good to make palettes etc based on the regions for consistency purposes
reg_ids = list(set(c.v.load_by_reg.reset_index()[c.GEO_COLS[0]].values))
reg_palette = {reg_ids[i]: IEA_PALETTE_16[i] for i in range(len(reg_ids))}

# Model palette
model_ids = list(set(c.v.load_by_reg.reset_index()['model'].values))
model_ids = [m for m in PRETTY_MODEL_NAMES if m in model_ids] + [m for m in model_ids if m not in PRETTY_MODEL_NAMES]
# Use extended palette so it can have more than 16 variables
model_palette = {model_ids[i]: IEA_PALETTE_16[i] for i in range(len(model_ids))}

# Regions and technologies will always be consistent this way. May need to be copied to other parts of the code
combined_palette = dict(STACK_PALETTE, **reg_palette, **model_palette)

[18:16:20 00:00] I:50:_drive_cache_wrapper - Computing objects: node_yr_df.
[18:16:30 00:09] I:57:_drive_cache_wrapper - Saved to objects cache: node_yr_df.parquet.


In [3]:
c.o.line_yr_df

[18:16:34 00:03] I:50:_drive_cache_wrapper - Computing objects: line_yr_df.
[18:16:43 00:08] I:57:_drive_cache_wrapper - Saved to objects cache: line_yr_df.parquet.


,name,property,timestamp,model,value,PLEXOSname,PLEXOS technology,WEO_tech,Operating_class,Region,Subregion,regFrom,regTo,InertiaLOW,InertiaHI,Cofiring,CCUS,IPP,CapacityCategory,Category,FlexCategory,StressPeriodType,ScaleCat,StorageDuration,VOM,FOM ($/kW),CAPEX ($/kW),RampCost
0,CR-ER_corridor,Export Limit,2035-12-31,01_China_2022_nMUT,41000.000000,CR-ER_corridor,Line,Line,Excluded,NaN,NaN,CR,ER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CR-ER_corridor,Flow,2035-12-31,01_China_2022_nMUT,11696.757837,CR-ER_corridor,Line,Line,Excluded,NaN,NaN,CR,ER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CR-ER_corridor,Flow Back,2035-12-31,01_China_2022_nMUT,285.116749,CR-ER_corridor,Line,Line,Excluded,NaN,NaN,CR,ER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CR-ER_corridor,Hours Congested,2035-12-31,01_China_2022_nMUT,1.000000,CR-ER_corridor,Line,Line,Excluded,NaN,NaN,CR,ER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CR-ER_corridor,Hours Congested Back,2035-12-31,01_China_2022_nMUT,0.000000,CR-ER_corridor,Line,Line,Excluded,NaN,NaN,CR,ER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
877,SGR-SWR_800kV_DC_3,Loss Back,2035-12-31,03_China_STEPS_2030_nMUT,2959.089301,SGR-SWR_800kV_DC_3,Line,Line,Excluded,NaN,NaN,SGR,SWR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
878,SGR-SWR_800kV_DC_3,Max Flow,2035-12-31,03_China_STEPS_2030_nMUT,0.000000,SGR-SWR_800kV_DC_3,Line,Line,Excluded,NaN,NaN,SGR,SWR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
879,SGR-SWR_800kV_DC_3,Min Flow,2035-12-31,03_China_STEPS_2030_nMUT,-5000.000000,SGR-SWR_800kV_DC_3,Line,Line,Excluded,NaN,NaN,SGR,SWR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
880,SGR-SWR_800kV_DC_3,Net Flow,2035-12-31,03_China_STEPS_2030_nMUT,-39065.086843,SGR-SWR_800kV_DC_3,Line,Line,Excluded,NaN,NaN,SGR,SWR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
